In [ ]:
#---#| default_exp protein.fasta

# Spectral Library from fasta

In [ ]:
from peptdeep.protein.fasta import *

In [ ]:
_lib = PredictSpecLibFasta(None, I_to_L=False, decoy='pseudo_reverse')
prot1 = 'MABCDEKFGHIJKLMNOPQRST'
prot2 = 'FGHIJKLMNOPQR'
protein_dict = {
    'xx': {
        'protein_id': 'xx',
        'gene_name': '',
        'sequence': prot1
    },
    'yy': {
        'protein_id': 'yy',
        'gene_name': 'gene',
        'sequence': prot2
    }
}
_lib.get_peptides_from_protein_dict(protein_dict)
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA
0,MABCDEK,0,0,True,False,,,7
1,LMNOPQR,0;1,0,False,True,,,7
2,LMNOPQRST,0,1,False,True,,,9
3,ABCDEKFGHIJK,0,1,True,False,,,12
4,MABCDEKFGHIJK,0,1,True,False,,,13
5,FGHIJKLMNOPQR,0;1,1,True,True,,,13
6,FGHIJKLMNOPQRST,0,2,False,True,,,15
7,ABCDEKFGHIJKLMNOPQR,0,2,True,False,,,19
8,MABCDEKFGHIJKLMNOPQR,0,2,True,False,,,20


In [ ]:
_lib.protein_df

,protein_id,gene_name,sequence
0,xx,,MABCDEKFGHIJKLMNOPQRST
1,yy,gene,FGHIJKLMNOPQR


In [ ]:
_lib.append_protein_name()
assert 'proteins' in _lib.precursor_df.columns
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,proteins,genes
0,MABCDEK,0,0,True,False,,,7,xx,
1,LMNOPQR,0;1,0,False,True,,,7,xx;yy,gene
2,LMNOPQRST,0,1,False,True,,,9,xx,
3,ABCDEKFGHIJK,0,1,True,False,,,12,xx,
4,MABCDEKFGHIJK,0,1,True,False,,,13,xx,
5,FGHIJKLMNOPQR,0;1,1,True,True,,,13,xx;yy,gene
6,FGHIJKLMNOPQRST,0,2,False,True,,,15,xx,
7,ABCDEKFGHIJKLMNOPQR,0,2,True,False,,,19,xx,
8,MABCDEKFGHIJKLMNOPQR,0,2,True,False,,,20,xx,


In [ ]:
#| hide
for i in range(len(_lib.precursor_df)):
    seq = _lib.precursor_df.sequence[i]
    # test is_prot_nterm
    if prot1.startswith(seq) or prot2.startswith(seq):
        assert _lib.precursor_df.is_prot_nterm[i], seq
    elif prot1[1:].startswith(seq): # M.xxxxx
        assert _lib.precursor_df.is_prot_nterm[i], seq
    else:
        assert not _lib.precursor_df.is_prot_nterm[i], seq
    # test is_prot_cterm
    if prot1.endswith(seq) or prot2.endswith(seq):
        assert _lib.precursor_df.is_prot_cterm[i], seq
    else:
        assert not _lib.precursor_df.is_prot_cterm[i], seq
    # test protein_idxes
    if seq in prot1 and seq in prot2:
        assert _lib.precursor_df.protein_idxes[i] == '0;1'
        assert _lib.precursor_df.proteins[i] == 'xx;yy'
        assert _lib.precursor_df.genes[i] == 'gene'
    else:
        assert ';' not in _lib.precursor_df.protein_idxes[i]
        assert ';' not in _lib.precursor_df.proteins[i]
        assert _lib.precursor_df.genes[i] == ''

In [ ]:
_lib.add_modifications()
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,proteins,genes
0,MABCDEK,0,0,True,False,Carbamidomethyl@C;Oxidation@M,4;1,7,xx,
1,MABCDEK,0,0,True,False,Carbamidomethyl@C,4,7,xx,
2,MABCDEK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Oxidat...,4;0;1,7,xx,
3,MABCDEK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term,4;0,7,xx,
4,LMNOPQR,0;1,0,False,True,Oxidation@M,2,7,xx;yy,gene
5,LMNOPQR,0;1,0,False,True,,,7,xx;yy,gene
6,LMNOPQRST,0,1,False,True,Oxidation@M,2,9,xx,
7,LMNOPQRST,0,1,False,True,,,9,xx,
8,ABCDEKFGHIJK,0,1,True,False,Carbamidomethyl@C,3,12,xx,
9,ABCDEKFGHIJK,0,1,True,False,Carbamidomethyl@C;Acetyl@Protein N-term,3;0,12,xx,


In [ ]:
#| hide
for i in range(len(_lib.precursor_df)):
    seq = _lib.precursor_df.sequence[i]
    mods = _lib.precursor_df.mods[i]
    sites = _lib.precursor_df.mod_sites[i]
    # test fix mods
    if 'C' in seq:
        assert str(seq.find('C')+1) in sites
        assert 'Carbamidomethyl@C' in mods
    else:
        assert 'Carbamidomethyl@C' not in mods
    # test Acetyl@Protein N-term
    if 'Acetyl@Protein N-term' in mods:
        assert _lib.precursor_df.is_prot_nterm[i]
        assert '0' in sites
    if '0' in mods:
        assert _lib.precursor_df.is_prot_nterm[i]
        assert 'Acetyl@Protein N-term' in mods
    if not _lib.precursor_df.is_prot_nterm[i]:
        assert 'Acetyl@Protein N-term' not in mods
    # test Oxidation@M
    if 'Oxidation@M' in mods:
        assert 'M' in seq
        assert str(seq.find('M')+1) in sites
    # test unmodified
    if mods == '':
        assert sites == ''
    if sites == '':
        assert mods == ''
df = _lib.precursor_df
# at least one nterm peptide does not contain Acetyl@Protein N-term
assert not df[df.is_prot_nterm].mod_sites.str.contains('0').all()
# at least one nterm peptide contains Acetyl@Protein N-term
assert df[df.is_prot_nterm].mod_sites.str.contains('0').any()
# test var mod Oxidation@M
assert not df[df.sequence.str.contains('M')].mods.str.contains('Oxidation@M').all()
assert df[df.sequence.str.contains('M')].mods.str.contains('Oxidation@M').any()
assert '' in df.mods.values

In [ ]:
_lib.special_mods = ['Phospho@S','Phospho@T']
_lib.add_special_modifications()
assert _lib.precursor_df.mods.str.contains('Phospho').any()
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,proteins,genes
0,MABCDEK,0,0,True,False,Carbamidomethyl@C;Oxidation@M,4;1,7,xx,
1,MABCDEK,0,0,True,False,Carbamidomethyl@C,4,7,xx,
2,MABCDEK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Oxidat...,4;0;1,7,xx,
3,MABCDEK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term,4;0,7,xx,
4,LMNOPQR,0;1,0,False,True,Oxidation@M,2,7,xx;yy,gene
5,LMNOPQR,0;1,0,False,True,,,7,xx;yy,gene
6,LMNOPQRST,0,1,False,True,Oxidation@M;Phospho@S,2;8,9,xx,
7,LMNOPQRST,0,1,False,True,Oxidation@M;Phospho@T,2;9,9,xx,
8,LMNOPQRST,0,1,False,True,Oxidation@M,2,9,xx,
9,LMNOPQRST,0,1,False,True,Phospho@S,8,9,xx,


In [ ]:
#| hide
_lib.add_peptide_labeling({
    'none': [], # not labelled for reference
    'light': ['Dimethyl@Any N-term','Dimethyl@K'],
    'heavy': ['Dimethyl:2H(6)13C(2)@Any N-term','Dimethyl:2H(6)13C(2)@K'],
})
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,proteins,genes,labeling_channel
0,MABCDEK,0,0,True,False,Carbamidomethyl@C;Oxidation@M,4;1,7,xx,,none
1,MABCDEK,0,0,True,False,Carbamidomethyl@C,4,7,xx,,none
2,MABCDEK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Oxidat...,4;0;1,7,xx,,none
3,MABCDEK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term,4;0,7,xx,,none
4,LMNOPQR,0;1,0,False,True,Oxidation@M,2,7,xx;yy,gene,none
...,...,...,...,...,...,...,...,...,...,...,...
115,MABCDEKFGHIJKLMNOPQR,0,2,True,False,Carbamidomethyl@C;Dimethyl:2H(6)13C(2)@Any N-t...,4;0;7;13,20,xx,,heavy
116,MABCDEKFGHIJKLMNOPQR,0,2,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Oxidat...,4;0;1;7;13,20,xx,,heavy
117,MABCDEKFGHIJKLMNOPQR,0,2,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Oxidat...,4;0;15;7;13,20,xx,,heavy
118,MABCDEKFGHIJKLMNOPQR,0,2,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Oxidat...,4;0;1;15;7;13,20,xx,,heavy


In [ ]:
from peptdeep.pretrained_models import ModelManager

In [ ]:
model_mgr = ModelManager(device='cpu')
model_mgr.load_installed_models()
model_mgr.verbose = False
_lib = PredictSpecLibFasta(
    model_mgr, I_to_L=False, 
    decoy='pseudo_reverse'
)
prot1 = 'MACDESTYKBKFGHIKLMNPQRST'
prot2 = 'FGHIKLMNPQR'
protein_dict = {
    'xx': {
        'protein_id': 'xx',
        'sequence': prot1
    },
    'yy': {
        'protein_id': 'yy',
        'sequence': prot2
    }
}
_lib.import_and_process_protein_dict(protein_dict)
_lib.generate_precursor_isotope = True
_lib.verbose = False
_lib.predict_all()
assert (_lib.precursor_df.decoy==1).any()
assert ('MACDESTY'[::-1]+'K') in _lib.precursor_df.sequence.values
assert 'isotope_apex_offset' in _lib.precursor_df.columns
assert 'isotope_apex_intensity' in _lib.precursor_df.columns
assert ~_lib.precursor_df.sequence.str.contains('B').any()
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,decoy,charge,...,isotope_apex_mz,isotope_right_most_mz,rt_pred,rt_norm_pred,ccs_pred,mobility_pred,frag_stop_idx,nce,instrument,frag_start_idx
0,LMNPQRST,0,1,False,True,Oxidation@M,2,8,0,2,...,481.739834,482.241484,0.021263,0.021263,318.941895,0.785035,7,30.0,Lumos,0
1,LMNPQRST,0,1,False,True,,,8,0,2,...,473.742377,474.244027,0.092409,0.092409,317.660034,0.781693,14,30.0,Lumos,7
2,ACDESTYK,0,0,True,False,Carbamidomethyl@C,2,8,0,2,...,487.200207,487.701857,0.032798,0.032798,329.176941,0.810355,21,30.0,Lumos,14
3,ACDESTYK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term,2;0,8,0,2,...,508.205490,509.208790,0.109105,0.109105,342.048767,0.842529,28,30.0,Lumos,21
4,SRQPNMLT,0,1,False,True,Oxidation@M,6,8,1,2,...,481.739834,482.241484,0.044289,0.044289,321.865784,0.792231,35,30.0,Lumos,28
5,SRQPNMLT,0,1,False,True,,,8,1,2,...,473.742377,474.244027,0.158330,0.158330,323.465607,0.795979,42,30.0,Lumos,35
6,YTSEDCAK,0,0,True,False,Carbamidomethyl@C,6,8,1,2,...,487.200207,487.701857,0.016275,0.016275,328.831970,0.809506,49,30.0,Lumos,42
7,YTSEDCAK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term,6;0,8,1,2,...,508.205490,509.208790,0.119288,0.119288,339.180786,0.835465,56,30.0,Lumos,49
8,MACDESTYK,0,0,True,False,Carbamidomethyl@C;Oxidation@M,3;1,9,0,2,...,560.717907,561.721207,0.048364,0.048364,351.815094,0.867675,64,30.0,Lumos,56
9,MACDESTYK,0,0,True,False,Carbamidomethyl@C,3,9,0,2,...,552.720450,553.723750,0.081848,0.081848,353.857971,0.872560,72,30.0,Lumos,64


In [ ]:
_lib.import_and_process_protein_dict(protein_dict)
_lib.add_peptide_labeling({
    'light': ['Dimethyl@Any N-term','Dimethyl@K'],
    'heavy': ['Dimethyl:2H(6)13C(2)@Any N-term','Dimethyl:2H(6)13C(2)@K'],
})
_lib.predict_all()
assert (_lib.precursor_df.decoy==1).any()
assert ('MACDESTY'[::-1]+'K') in _lib.precursor_df.sequence.values
assert 'isotope_apex_offset' in _lib.precursor_df.columns
assert 'isotope_apex_intensity' in _lib.precursor_df.columns
assert ~_lib.precursor_df.sequence.str.contains('B').any()
_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,decoy,charge,...,isotope_apex_mz,isotope_right_most_mz,rt_pred,rt_norm_pred,ccs_pred,mobility_pred,frag_stop_idx,nce,instrument,frag_start_idx
0,LMNPQRST,0,1,False,True,Oxidation@M;Dimethyl@Any N-term,2;0,8,0,2,...,495.755484,496.257134,0.242660,0.242660,345.390869,0.850475,7,30.0,Lumos,0
1,YTSEDCAK,0,0,True,False,Carbamidomethyl@C;Acetyl@Protein N-term;Dimeth...,6;0;8,8,1,2,...,526.243325,526.744975,0.106988,0.106988,347.019043,0.855165,14,30.0,Lumos,7
2,YTSEDCAK,0,0,True,False,Carbamidomethyl@C;Dimethyl:2H(6)13C(2)@Any N-t...,6;0;8,8,1,2,...,523.275878,523.777528,0.009153,0.009153,331.465332,0.816775,21,30.0,Lumos,14
3,SRQPNMLT,0,1,False,True,Dimethyl:2H(6)13C(2)@Any N-term,0,8,1,2,...,491.780212,492.281862,0.152593,0.152593,320.333069,0.788686,28,30.0,Lumos,21
4,SRQPNMLT,0,1,False,True,Oxidation@M;Dimethyl:2H(6)13C(2)@Any N-term,6;0,8,1,2,...,499.777669,500.279319,0.040845,0.040845,319.400391,0.786564,35,30.0,Lumos,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,FGHIKLMNPQRST,0,2,False,True,Dimethyl@Any N-term;Dimethyl@K,0;5,13,0,2,...,792.937585,793.940885,0.636318,0.636318,428.658142,1.060983,792,30.0,Lumos,780
84,FGHIKLMNPQRST,0,2,False,True,Dimethyl@Any N-term;Dimethyl@K,0;5,13,0,3,...,528.960816,529.629682,0.636318,0.636318,482.273010,0.795796,804,30.0,Lumos,792
85,FGHIKLMNPQRST,0,2,False,True,Dimethyl:2H(6)13C(2)@Any N-term;Dimethyl:2H(6)...,0;5,13,0,3,...,534.323729,534.992596,0.206957,0.206957,478.660187,0.789903,816,30.0,Lumos,804
86,SRQPNMLKIHGFT,0,2,False,True,Oxidation@M;Dimethyl@Any N-term;Dimethyl@K,6;0;8,13,1,2,...,800.935042,801.938342,0.468698,0.468698,428.150787,1.059819,828,30.0,Lumos,816
